# Blackjack AR

In [ ]:
#cartas de poker
class Carta:
    def __init__(self, palo, valor, imagen):
        self.palo = palo
        self.valor = valor
        self.imagen = imagen
        self.enJuego = False

    def __str__(self):
        return f"{self.valor} de {self.palo}"

In [ ]:
def init_cartas():
    palos = ['clubs', 'diamonds', 'heards', 'spades']
    valores = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
    imagenes = []
    for palo in palos:
        for valor in valores:
            imagenes.append(f"cartas/{valor}{palo[0].upper()}.png")
    return [Carta(palo, valor, imagen) for palo in palos for valor in valores for imagen in imagenes]

In [ ]:
import cv2
import numpy as np
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

#Capturar video desde la cámara
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
   
    # Mostrar el frame con el escudo, bufanda y marcas de colores
    cv2.imshow("Detección de Rostros con Bufanda y Marcas", frame)

    # Salir si se presiona escape
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()